In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load SpaceX dataset
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Min and max payload
min_payload = spacex_df["Payload Mass (kg)"].min()
max_payload = spacex_df["Payload Mass (kg)"].max()

# Initialize the app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # Dropdown for site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder="Select a Launch Site",
        searchable=True
    ),
    
    # RangeSlider for payload
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        value=[min_payload, max_payload]
    ),
    
    # Pie chart
    dcc.Graph(id='success-pie-chart'),
    
    # Scatter plot
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback function for interactive plots
@app.callback(
    [Output('success-pie-chart', 'figure'),
     Output('success-payload-scatter-chart', 'figure')],
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def get_plots(entered_site, payload_range):
    filtered_df = spacex_df[(spacex_df["Payload Mass (kg)"] >= payload_range[0]) & 
                            (spacex_df["Payload Mass (kg)"] <= payload_range[1])]

    if entered_site == 'ALL':
        # Pie chart for all sites
        fig1 = px.pie(filtered_df, 
                      names='Launch Site', 
                      title='Total Successful Launches for All Sites')
        # Scatter plot for all sites
        fig2 = px.scatter(filtered_df, 
                          x="Payload Mass (kg)", 
                          y="class", 
                          color="Booster Version Category",
                          title="Payload Mass vs. Launch Outcome for All Sites")
    else:
        # Filter data for selected site
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        # Pie chart for the selected site
        fig1 = px.pie(site_df, 
                      names='class', 
                      title=f'Total Successful Launches for {entered_site}',
                      color_discrete_sequence=["red", "blue"])
        # Scatter plot for the selected site
        fig2 = px.scatter(site_df, 
                          x="Payload Mass (kg)", 
                          y="class", 
                          color="Booster Version Category",
                          title=f"Payload Mass vs. Launch Outcome for {entered_site}")

    return fig1, fig2

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8088)  # Changed to port 8051 (or any other available port)
